In [ ]:
import re
import nrrd
import os, glob
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from patchify import patchify, unpatchify
from ipywidgets import interact, interactive, IntSlider, ToggleButtons

import loss
import Helper
import allMetrics

os.environ["CUDA_VISIBLE_DEVICES"]="2"

access_token = '5307815831:AAEKVzHQFAVWMwXseU9L1yKKtsBq4YPZM8Y'
chat_id = '503932462'

# /home/tester/jianhoong/ # ntu
# data_dir = /media/dro/JHSeagate/FYP/ # dgx
data_dir = '/media/dro/JHSeagate/FYP/jh_fyp_work/ct_scans_data/raw_data/'

z_train_image = os.path.join(data_dir, 'train_data/training_images/')
z_valid_image = os.path.join(data_dir, 'valid_data/valid_images/')
z_test_image = os.path.join(data_dir, 'test_data/testing_images/')

z_train_mask = os.path.join(data_dir, 'train_data/training_masks/')
z_valid_mask = os.path.join(data_dir, 'valid_data/valid_masks/')
z_test_mask = os.path.join(data_dir, 'test_data/testing_masks/')

def sorted_alnum(l):
    def convert(text): return int(text) if text.isdigit() else text
    def alphanum_key(key): return [convert(c)
                                   for c in re.split('([0-9]+)', key)]
    return sorted(l, key=alphanum_key)

# train_data_with_96 = [15, 16, 20, 21, 23, 33, 37, 52, 64, 69, 72, 76, 77, 83, 92, 94, 95,
#                       97, 103, 116, 123, 129, 157, 159, 162, 163, 170, 171, 180, 191, 192, 198, 200, 207, 209]
# valid_data_with_96 = [213, 219, 221, 227, 231,
#                       238, 240, 243, 249, 250, 255, 256, 257, 270]
# test_data_with_96 = [271, 272, 274, 277, 283,
#                      286, 287, 293, 296, 300, 303, 305, 306, 314]

train_data_with_96 = [1, 2, 7, 11, 15, 16, 20, 21, 
                      23, 32, 33, 37, 39, 42, 52, 61, 
                      64, 65, 67, 69, 72, 76, 77, 83, 
                      92, 94, 95, 97, 98, 103, 115, 
                      116, 123, 124, 125, 127, 129, 
                      130, 141, 154, 157, 158, 159, 
                      162, 163, 167, 168, 170, 171, 
                      177, 180, 181, 183, 191, 192, 
                      197, 198, 200, 204, 207, 209]


valid_data_with_96 = [211, 212, 213, 219, 221, 222, 
                      225, 227, 231, 238, 240, 242, 
                      243, 244, 245, 249, 250, 251, 
                      255, 256, 257, 259, 267, 268, 
                      269, 270]

test_data_with_96 = [271, 272, 274, 277, 279, 283, 
                     286, 287, 293, 296, 300, 301, 
                     303, 305, 306, 314, 321, 323, 
                     330, 
336]

train_path = sorted_alnum([os.path.join(z_train_image, file) for file in os.listdir(
    z_train_image) if int(re.findall(r'\d+', file)[0]) in train_data_with_96])
train_mask_path = sorted_alnum([os.path.join(z_train_mask, file) for file in os.listdir(
    z_train_mask) if int(re.findall(r'\d+', file)[0]) in train_data_with_96])

valid_path = sorted_alnum([os.path.join(z_valid_image, file) for file in os.listdir(
    z_valid_image) if int(re.findall(r'\d+', file)[0]) in valid_data_with_96])
valid_mask_path = sorted_alnum([os.path.join(z_valid_mask, file) for file in os.listdir(
    z_valid_mask) if int(re.findall(r'\d+', file)[0]) in valid_data_with_96])

test_path = sorted_alnum([os.path.join(z_test_image, file) for file in os.listdir(
    z_test_image) if int(re.findall(r'\d+', file)[0]) in test_data_with_96])
test_mask_path = sorted_alnum([os.path.join(z_test_mask, file) for file in os.listdir(
    z_test_mask) if int(re.findall(r'\d+', file)[0]) in test_data_with_96])

# model_checkpoint_path = 'E:/FYP/jh_fyp_work/3D_UNet/Final/SavedModels/Model10.hdf5'
model_checkpoint_path = '/media/dro/JHSeagate/FYP/jh_fyp_work/3D_UNet/Final/SavedModels/3mm_5mm.hdf5'

model = tf.keras.models.load_model(model_checkpoint_path, custom_objects = {                                                                
                                                                                'dice_coef':allMetrics.dice_coef,
                                                                                   'tversky_crossentropy':loss.tversky_crossentropy})

input_shape = (64,64,96)
num_class = 1
desired_step = 32

test_patient_img = Helper.process_scan(valid_path[0])
test_patient_msk = Helper.read_nrrd_file(valid_mask_path[0])

test_patch = patchify(test_patient_img, input_shape, step = desired_step) # Yield 15 x 15 x 3 of size 64 x 64 x 32 cubes
test_mask_patch = patchify(test_patient_msk, input_shape, step = desired_step)

test_patient_cubes = np.reshape(test_patch, (-1, test_patch.shape[3], test_patch.shape[4], test_patch.shape[5])) # Collapse 15 x 15 x 3 into 675 cubes x 64 x 64 x 32
test_patient_mask_cubes = np.reshape(test_mask_patch, (-1, test_mask_patch.shape[3], test_mask_patch.shape[4], test_mask_patch.shape[5])) # 675 x 64 x 64 x 32

test_patient_input = np.expand_dims(test_patient_cubes, axis=4)
test_patient_mask_input = np.expand_dims(test_patient_mask_cubes, axis=4)

test_patient_pred = model.predict(test_patient_input)

test_patient_pred_reshaped = np.reshape(test_patient_pred, 
                                        (test_patch.shape[0], test_patch.shape[1], test_patch.shape[2],
                                         test_patch.shape[3], test_patch.shape[4], test_patch.shape[5]))

reconstruct_pred = unpatchify(test_patient_pred_reshaped, test_patient_img.shape)
reconstruct_pred[reconstruct_pred < 1] = 0

@interact
def explore_prediction(layer = (0,95), view = ["axial"]):
    if view == 'axial':
        img_array_view = test_patient_img[:, :, layer]
        msk_array_view = test_patient_msk[:, :, layer]
        pred_array_view = reconstruct_pred[:, :, layer]
# , cmap='gray'
    plt.figure(figsize=(18, 6))
    plt.subplot(1, 3, 1)
    plt.imshow(img_array_view, cmap = "gray", aspect = "auto")
    plt.title('Img', fontsize=10)
    plt.subplot(1, 3, 2)
    plt.imshow(msk_array_view, cmap = "gray",aspect = "auto")
    plt.title('Mask', fontsize=10)
    plt.subplot(1, 3, 3)
    plt.imshow(pred_array_view, cmap = "gray",aspect = "auto")
    plt.title('Prediction', fontsize=10)
    plt.axis('off')
    plt.show()
    
headerInfo = nrrd.read_header(valid_path[0])

modelname = 'run1_52epochs'
pred_volume_file_location = f"/media/dro/JHSeagate/FYP/jh_fyp_work/ct_scans_data/prediction_data/3mm_5mm_Predictions/{modelname}.nrrd"
nrrd.write(pred_volume_file_location, reconstruct_pred, headerInfo)